In [2]:
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models
import os
import gensim
import nltk
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [3]:
df = pd.read_csv('1.csv')

In [4]:
new_column_names = {old_name: str(i + 1) for i, old_name in enumerate(df.columns)}

# Apply the new column names to the DataFrame
df_renamed = df.rename(columns=new_column_names)

In [5]:
columns_to_keep_1 = [
    "38"
]
df1 = df_renamed[columns_to_keep_1]

In [6]:
df1 = df1.dropna()

In [7]:
# Load standard stop words
stop_words = set(stopwords.words('english'))

# Add custom stop words
custom_stop_words = [
    'use','want', 'and', 'the', 'i', 'changed','changes', 
    'stay', 'one', 'said', 'say','to','is','was','for','in','of','The','but','at','a','an','as','had','have','on','from','you','would','will','not','us','we','get','could'  # Ensure not to duplicate words
]
stop_words.update(custom_stop_words)

# Function to generate tokens
def generate_tokens(text):
    # Tokenize and filter out stop words and any other potential noise
    return [word for word in word_tokenize(text.lower()) if word not in stop_words and word.isalpha()]

# Apply the tokenization function to the 'Review' column
df1['tokens'] = df1['38'].apply(generate_tokens)

In [8]:
def create_dictionary(words):
    return corpora.Dictionary(words)
id2word=create_dictionary(df1['tokens'])

In [9]:
def create_document_matrix(tokens,id2word):
    corpus = []
    for text in tokens:
        corpus.append(id2word.doc2bow(text))
    return corpus
#passing the dataframe column having tokens and dictionary
corpus=create_document_matrix(df1['tokens'],id2word)
print(df1['tokens'][0])
print(corpus[0])

['due', 'fact', 'people', 'need', 'home', 'time', 'epidemic', 'desire', 'buy', 'clothes', 'decreased', 'inclined', 'buy', 'comfortable', 'durable', 'clothes']
[(0, 2), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]


In [10]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=10,
                                            random_state=100,
                                             ) #building up LDA momdel

In [11]:
#applying LDA model
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis 

C:\Users\nevea\anaconda3\Lib\site-packages\sklearn\manifold\_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.156481  0.001791       1        1  15.448701
7      0.003594  0.080104       2        1  12.834215
6     -0.112328  0.135129       3        1  12.574264
3      0.113527 -0.148806       4        1  12.267315
1      0.145701 -0.018469       5        1  11.528645
9      0.137985  0.086371       6        1  10.018527
0      0.026808  0.168951       7        1   9.436209
2     -0.010728 -0.139902       8        1   6.609021
4     -0.108220 -0.126942       9        1   4.797444
5     -0.039857 -0.038227      10        1   4.485660, topic_info=        Term       Freq      Total Category  logprob  loglift
21    change  11.000000  11.000000  Default  30.0000  30.0000
0        buy  27.000000  27.000000  Default  29.0000  29.0000
29     items   9.000000   9.000000  Default  28.0000  28.0000
93   fashion   5.000000   5.000000  Default  27.0000  27.0000
52      like   9.000000   9.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
0        buy   0.552091  27.890362  Topic10  -4.7160  -0.8180
52      like   0.552048   9.696136  Topic10  -4.7160   0.2384
94      fast   0.552040   4.358880  Topic10  -4.7161   1.0379
6    durable   0.552026   3.769348  Topic10  -4.7161   1.1832
115     many   0.552022   5.385590  Topic10  -4.7161   0.8264

[571 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
610       7  0.716049     aaligned
413       6  0.748662  accentuates
487       9  0.872220  accessories
419       8  0.753027  adventurous
133       3  0.567052     affected
...     ...       ...          ...
329       3  0.584763      working
329       8  0.292381      working
506       4  0.702192     workwear
254       1  0.687541          yes
62        8  0.753873         zara

[693 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 8, 7, 4, 2, 10, 1, 3, 5, 6])

In [12]:
# analyse csr report topic            
# Standard NLTK stop words
stop_words = set(stopwords.words('english'))

# Add custom stop words
custom_stop_words = [
    'use','want', 'and', 'the', 'i', 'changed','changes', 
    'stay', 'one', 'said', 'say','to','is','was','for','in','of','The','but','at','a','an','as','had','have','on','from','you','would','will','not','us','we','get','could'] # Add more as needed
stop_words.update(custom_stop_words)

# Apply LDA
num_topics = 10  
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=id2word, passes=15)

# Print the topics
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))


Topic: 0 
Words: 0.022*"clothes" + 0.021*"money" + 0.021*"spend" + 0.019*"increased" + 0.019*"wardrobe" + 0.019*"vinted" + 0.013*"items" + 0.013*"buy" + 0.013*"quality" + 0.013*"due"
Topic: 1 
Words: 0.026*"shopping" + 0.017*"made" + 0.009*"think" + 0.009*"many" + 0.009*"shop" + 0.009*"home" + 0.009*"since" + 0.009*"focused" + 0.009*"comfort" + 0.009*"purchases"
Topic: 2 
Words: 0.029*"clothes" + 0.024*"purchase" + 0.018*"work" + 0.018*"buy" + 0.018*"offline" + 0.018*"online" + 0.012*"much" + 0.012*"clothing" + 0.012*"casual" + 0.012*"things"
Topic: 3 
Words: 0.037*"change" + 0.035*"pandemic" + 0.032*"clothing" + 0.028*"purchases" + 0.021*"shopping" + 0.016*"covid" + 0.014*"online" + 0.014*"buying" + 0.014*"deals" + 0.014*"good"
Topic: 4 
Words: 0.049*"clothes" + 0.030*"less" + 0.028*"buy" + 0.023*"spend" + 0.021*"much" + 0.015*"need" + 0.015*"clothing" + 0.014*"money" + 0.011*"pandemic" + 0.009*"items"
Topic: 5 
Words: 0.040*"clothes" + 0.037*"buy" + 0.020*"buying" + 0.020*"online" + 